In [1]:
import re
from datetime import datetime
import xml.etree.ElementTree as ET

In [3]:
tree = ET.parse('volumes/frus1969-76v30.xml')
root = tree.getroot()

In [ ]:
# remove namespace
# year: frus:doc-dateTime-max
# sent from: <placeName>New York</placeName>
# volume from file name
# from: <gloss type="from"> or <persName type="from">
# to: <gloss type="to"> or <persName type="to">
# source: <... type="source" ..>

# ignore all <note/>

In [ ]:
person_item_list = root.findall("text/front/div/[@{http://www.w3.org/XML/1998/namespace}id='persons']")[0].findall('.//item')
person_dict = {}

def extract_person(person_item):
    persName_item = person_item.find('.//persName')
    person_name = persName_item.text
    person_id = persName_item.attrib['{http://www.w3.org/XML/1998/namespace}id']

    all_text = "".join(person_item.itertext())
    end_idx = all_text.find(person_name) + len(person_name+',')
    person_descp = " ".join(all_text[end_idx:].split())

    person_name = " ".join(re.sub(',',''," ".join(person_name.split(', ')[::-1])).split())

    person_dict[person_id] = {'name':person_name, 'descp':person_descp}
    return

for item in root.findall("text/front/div/[@{http://www.w3.org/XML/1998/namespace}id='persons']")[0].findall('.//item'):
    extract_person(item)

In [ ]:
person_dict

## done above ^
## construction below 

In [ ]:
# this function handles file's sender and receiver
def process_from_to_types(from_list):

    processed_list = []

    for item in from_list:
        
        if item.tag == 'persName' and 'corresp' in item.attrib.keys():
            person_id = item.attrib['corresp']
            processed_list.append(person_id)
        elif item.tag == 'persName':
            processed_list.append(item.text)
        elif item.tag == 'gloss':
            processed_list.append(item.text)
        else:
            raise NotImplementedError('from type tag unidentified')
    
    return processed_list

In [ ]:
subseries_id = root.find('teiHeader/fileDesc/titleStmt/title[@type="sub-series"]').text
volume_id = root.find('teiHeader/fileDesc/titleStmt/title[@type="volume-number"]').text

doc_dict = {}

def process_document(doc_elem):

    doc_id = doc_elem.attrib['{http://www.w3.org/XML/1998/namespace}id']
    doc_date = doc_elem.attrib["{http://history.state.gov/frus/ns/1.0}doc-dateTime-max"].split('T')[0]
    doc_date = datetime.strptime(doc_date, '%Y-%m-%d')

    doc_sent_place = doc_elem.find('.//placeName')
    if doc_sent_place is not None:
        doc_sent_place = doc_sent_place.text

    from_list = doc_elem.findall('head//*[@type="from"]')
    to_list = doc_elem.findall('head//*[@type="to"]')

    doc_sent_from = process_from_to_types(from_list)
    doc_sent_to = process_from_to_types(to_list)

    doc_source = doc_elem.find('.//*[@type="source"]')
    if doc_source is not None:
        doc_source = " ".join("".join(doc_source.itertext()).split())

    doc_dict["-".join([subseries_id,volume_id,doc_id])] = {'subseries':subseries_id, 'volume':volume_id, 'doc_id':doc_id,'date':doc_date, 'place':doc_sent_place, 'from':doc_sent_from, 'to':doc_sent_to, 'source':doc_source}

    return

for doc_elem in root.findall('text/body/div//div[@type="document"]'):
    process_document(doc_elem)

In [ ]:
doc_dict

In [ ]:
place = []

for k,v in doc_dict.items():
    place+=(v['to'])

In [ ]:
from collections import Counter

Counter(place)